# 함수 II
1. 일급 객체
2. 중첩 함수
3. 익명 함수
4. 제너레이터
5. 재귀 함수

---

## 1. 일급 객체
- first class object, first class citizen
- 파이썬에서는 함수도 일급 객체다
    - **파이썬에서는 함수도 아래의 역할이 주어졌다**
- 일급 객체의 조건 (아래 3가지 조건을 만족)
    1. 함수의 인자로 전달된다  
        def fx(func): => 인자가 함수
    2. 함수의 반환값이 된다  
        def fx(func):
            return func => 함수를 반환 가능
    3. 수정, 할당이 된다  
        var = fx() => 반환값이 함수였을 때 변수 할당 가능

In [1]:
def answer():
    print(42)
    
def run_sth(func):
    func() # ()가 붙으면 실행한다는 뜻 
    # 함수를 인자로 받고 실행함

run_sth(answer)

42


In [2]:
def add_args(arg1, arg2):
    print(arg1 + arg2)

def run_sth2(func, *args): # asterisk 패킹 사용
    func(*args) 

run_sth2(add_args, 3, 5)    

8


---

## 2. 중첩 함수
- 일급객체를 이용해 함수 안에서 또 다른 함수를 정의하는 것
- 왜 이렇게 쓰나? **내부함수의 캡슐화**
    - 메모리 절약할 수 있음
    - 변수가 섞여서 불필요하게 충돌하는 것을 방지함
    - 목적에 맞게 변수를 그룹화하여 관리, 책임을 명확히함

In [9]:
def outer(a, b):
    def inner(c, d):
        return c + d
    return inner(a, b)

outer(1, 1) # outer => 외부함수, inner => 내부함수

2

In [8]:
# 함수 내의 함수에 넣으면 오류
# 내부 함수이기 때문에 건들일 수 없음
inner(1, 1)

NameError: name 'inner' is not defined

In [10]:
def knight(saying):
    def inner():
        return f'We are the knights who say: {saying}'
    return inner

a = knight('hi')
b = knight('안녕')

In [13]:
a, b # ()없이

(<function __main__.knight.<locals>.inner()>,
 <function __main__.knight.<locals>.inner()>)

In [14]:
a(), b() # ()있이 => 함수 실행

('We are the knights who say: hi', 'We are the knights who say: 안녕')

### (1) 클로저 | closure
- 이와 같이 def inner(): 처럼 인자가 없어도 외부함수의 인자를 **참조**할 수 있다  
=> **클로저** (중첩함수의 일부분)
- 수정이나 활용은 불가능
- 아래 3가지 조건을 충족 => **클로저**
    1. 중첩함수일 것
    2. 내부함수가 외부함수의 상태값을 참조할 것  
    We are the knights who say: {saying}
    3. 외부함수의 리턴값이 내부함수일 것  
    return inner
- 정의 : 외부함수의 상태값을 기억하는 함수 (호출 시 사용 가능)

In [21]:
# 재활용을 할 수 있는 장점이 있다
def multiply(x):
    def inner(y):    # 1.
        return x * y # 2.
    return inner     # 3.
# 클로저의 모든 조건 충족

m = multiply(5)
n = multiply(6)

In [28]:
m, n

(<function __main__.multiply.<locals>.inner(y)>,
 <function __main__.multiply.<locals>.inner(y)>)

In [23]:
m(10), n(10)

(50, 60)

In [29]:
del(multiply) # multiply 메모리 지우기
multiply

NameError: name 'multiply' is not defined

In [31]:
# multiply를 지워도 객체를 따로 기억해 실행 가능하다 => 메모리 효율적 사용 가능
m(8)

40

### - 클로저 함수 실습

In [55]:
def add(a, b):
    return a + b

def square(func):
    def inner(a, b): # func의 인자가 둘 => inner에서 설정함.
        result = func(a, b)
        return result * result
    return inner

a = square(add)
a(2, 6)

64

### (2) 데코레이터
- 위의 과정을 거치지 않고 더 편리하게 하는 방법
- 메인 함수에 또 다른 함수를 취해 반환할 수 있게 함
- 재사용성이 높음
- 가독성, 직관성이 좋음

In [57]:
@square
def plus(a, b):
    return a + b

plus(4, 5) # square를 탄다.

81

### (3) 범위 | scope
- 지금까지 배운 범위 
    - 전역: global
    - 지역: local
- nonlocal이라는 범위가 생김

In [58]:
a = 1 # global
def plus(a, b):
    x = 2 # local
    return a + b

In [63]:
# local을 쓰고 싶을 때 nonlocal 활용
a = 3 # global

def outer(c):              # c = 9
    b = 5 # local
    def inner():           # c = 9
        c = 999 # nonlocal # c = 999로 바뀜
        return c
    return inner()

outer(9)

999

In [65]:
def outer(c):              
    b = 5 # local
    def inner():           
        c += 1   # 오류
        return c
    return inner()

outer(9)

UnboundLocalError: local variable 'c' referenced before assignment

In [66]:
def outer(c):              
    b = 5 # local
    def inner():
        nonlocal c # c의 범위를 바꿔줘야함
        c += 1   
        return c
    return inner()

outer(9)

10

### - 실습
- fx1: speed, limit를 받아서 내 속도가 제한속도를 위반하는지 t/f
- fx2: 클로저, 초과할 경우 얼마나 초과하는지 프린트하는 함수
- 실행은 데코레이터로 fx1를 실행하면 되도록 만들기

In [69]:
# fx2
def violate(func):
    def inner(speed, limit):
        if func(speed, limit):
            return f'{speed - limit}km/h 초과'
        else:
            return '정상 속도'
    return inner

# fx1
@violate
def is_speeding(speed, limit):
    return speed > limit

In [70]:
is_speeding(100, 80)

'20km/h 초과'

In [71]:
is_speeding(50, 80)

'정상 속도'

---

## 3. 익명 함수 | lambda
- 이름이 없다 => **예약어가 없다**  
  def is_speeding():  
         return
- def, return => 이런 예약어를 안 씀
- is_speeding => 이름은 써도 되고 안 써도 됨
- 익명 함수 왜 씀?
    - 재사용할 일이 없고 빨리 처리해야 할 때 **단순한 용도의 함수**가 필요한 경우 사용
- 잦은 사용은 권하지 않음
- 형식
    - **lambda 인자: 요리할 코드**

In [72]:
def add_one(x):
    return x + 1

# 위의 함수를 단순하게 사용할 수 있음
(lambda x: x + 1)(2)

3

In [73]:
# 이름 만들기, 변수 여러 개
f = lambda x, y: x + y
f(3, 5)

8

### - 람다 함수 실습
- 단어가 들어왔을 때 첫 글자를 대문자로 바꾸고 단어 끝에 !를 붙이도록 람다를 만들기
- 예: hello -> Hello!

In [74]:
(lambda word: word.capitalize() + '!')('hello')

'Hello!'

---

## 4. 제너레이터
- return 대신 yield를 사용
- 시퀀스를 순회할 때 시퀀스를 생성하는 객체
    - [1, 2, 3]일 때 return은 한 번에 반환한다면  
    yield는 **각각 반환**
- **한 번 사용되고 사라짐** => 메모리 효율이 좋음

In [75]:
def print_number(num):
    for i in range(num):
        yield i
        
print_number(10)

<generator object print_number at 0x0000019CD70CB3C0>

In [77]:
fx = print_number(5)

# 시퀀스 객체이기 때문에 순회를 해야 나옴
for i in fx:
    print(i)

0
1
2
3
4


In [79]:
for i in fx:
    print(i)
# 사용하고 메모리에서 사라졌기 때문에 재사용하면 아무것도 안 나옴

### - 제너레이터 실습
- range() 구현하기
- 제너레이터 사용
- def my_range(start, end, step):  
          yield
- ranger = my_range(a, b, c)

In [83]:
def my_range(end, start=0, step=1):
    while start < end:
        yield start
        start += step

In [84]:
ranger = my_range(5)
for i in ranger:
    print(i)

0
1
2
3
4


In [86]:
# generator 더 쉽게 만들기
# 리스트 컴프리헨션에서 []가 아닌 ()로
ranger = (i for i in range(5))
ranger

<generator object <genexpr> at 0x0000019CD70D1200>

In [87]:
for i in ranger:
    print(i)

0
1
2
3
4


---

## 5. 재귀 함수
- 너무 깊으면 예외 발생 => 주의
- 자기 자신을 호출하는 함수
- [[1, 2, 3], [[1, 1]], 4, 5] -> [1, 2, 3, 1, 1, 4, 5]
- 차원이 다른 모든 요소를 하나 차원의 리스트에 담고 싶을 때

In [89]:
isinstance(4, int) # 4가 int인가? t/f 반환

True

In [90]:
def flatten(sent):
    for word in sent:
        if isinstance(word, list):
            # true
            for sub_word in flatten(word): # 자기 자신 호출
                yield sub_word
        else:
            # false
            yield word

In [91]:
a = [[1, 2, 3], [[1, 1]], 4, 5]
flatten(a)

<generator object flatten at 0x0000019CD70D1900>

In [92]:
for i in flatten(a):
    print(i)

1
2
3
1
1
4
5


In [93]:
# python 3.3부터 가능한 기능
def flatten(sent):
    for word in sent:
        if isinstance(word, list):
            # for sub_word in flatten(word):
            #     yield sub_word
            yield from flatten(word) # 위 두 줄 축약
        else:
            yield word

---
---

# 예외 처리 | exception handling
- 예외 => 오류
- 목적 : 프로그램 정상 종료
- 예외 발생 시, 사용자에게 알리고 조치를 취함
- 소프트랜딩

In [94]:
# 1. Division Error
# 0으로 나눌 수 없음
1 / 0     

ZeroDivisionError: division by zero

In [99]:
# 2. Value Error
# 형변환 불가
int('ssss')     

ValueError: invalid literal for int() with base 10: 'ssss'

In [100]:
# 3. name Error
# 정의하지 않은 변수
hello += 1

NameError: name 'hello' is not defined

In [101]:
# 4. index Error
# 없는 인덱스
'ssss'[10]

IndexError: string index out of range

In [102]:
# 예외 처리 방법
try:
    # <오류 발생될 법한 코드 블럭>
    10 / 0
except ZeroDivisionError: # <에러타입>:
    # <처리할 방법>
    print('0으로 나눌 수 없음')

0으로 나눌 수 없음


In [103]:
# 나머지는 실행하지 않고 error만 프린트하고 끝
try:
    for i in range(10):
        print(10 / i)
except ZeroDivisionError:
    print('error')

error


In [104]:
# except 위치 중요
for i in range(10):
    try:
        print(10 / i)
    except ZeroDivisionError:
        print('error')

error
10.0
5.0
3.3333333333333335
2.5
2.0
1.6666666666666667
1.4285714285714286
1.25
1.1111111111111112


In [107]:
# input 받는 상황 오류 대처
word = 'hello'

while True:
    index = input('인덱스를 입력하세요> ')
    if index == 'q':
        break
        
    try:
        print(word[int(index)])
    except IndexError as e1: # handler
        print('index error')
        print(e1)
    except ValueError as e2:
        print('type error')
        print(e2)

인덱스를 입력하세요>  8


index error
string index out of range


인덱스를 입력하세요>  ddd


type error
invalid literal for int() with base 10: 'ddd'


인덱스를 입력하세요>  q


---

# 예외 발생시키기 
- 프로그램을 종료하고자 할 때
1. raise
2. assert

## (1) raise
- raise ValueError('print ...')

In [108]:
while True:
    num = input('number >> ')
    # 숫자가 아니면 error를 발생시키겠다
    if not num.isdigit():
        raise ValueError('숫자가 아닙니다.')
    else:
        print(num)
        break

number >>  d


ValueError: 숫자가 아닙니다.

## (2) assert
- assert <참인 조건>, '예외 메시지' 
- 그랬을 때 AssertionError 발생
- **check 기능**

In [110]:
def get_binary(num):
    # 내가 낸 에러이기 때문에 어디에서 에러가 뜨는지 알 수 있는 장점이 있음
    assert isinstance(num, int), '정수 아님'
    return bin(num)

get_binary('ee')

AssertionError: 정수 아님

---

# 예외 정의하기
- 내가 원하는 에러를 발생시키고 싶을 때
- 사용자 정의 예외
- Exception이라는 부모 클래스를 상속받아야 한다

In [113]:
# 내용은 안 써도 되는데 꼭 Exception 클래스를 상속 받아야 함
class MyException(Exception):
    pass

In [115]:
for word in ['a', 'b', 'C']:
    if word.isupper():
        raise MyException('대문자 안 됨!')
    else:
        print(word)

a
b


MyException: 대문자 안 됨!

In [117]:
class UppercaseMyException(Exception):
    def __init__(self):
        super().__init__('대문자 안 됨ㅠ')
        
        
for word in ['a', 'b', 'C']:
    if word.isupper():
        raise UppercaseMyException
    else:
        print(word)

a
b


UppercaseMyException: 대문자 안 됨ㅠ